# **Armin Strom website scraping **

importing nedded  libraries :

In [2]:
from bs4 import BeautifulSoup # For HTML parsing
import requests # Website connections
from time import sleep # To prevent overwhelming the server between connections
from collections import Counter # Keep track of our term counts
import pandas as pd # For converting results to a dataframe and bar chart plots
import json # For parsing json
from bs4 import BeautifulSoup
%matplotlib inline
from lxml import etree # For xpath
import numpy as np
import re

In [3]:
# links for all the parent module:
watch_url = []
parents = ["https://arminstrom.com/en/collection/system-78/?v=ee2d312f5477","https://arminstrom.com/en/collection/resonance/?v=ee2d312f5477#pure", 'https://arminstrom.com/en/collection/masterpiece-collection/?v=ee2d312f5477']
prefix = 'https://arminstrom.com/'
for parent in parents:
  url = parent
  r = requests.get(url)
  soup = BeautifulSoup(r.text, 'html.parser')

  links = soup.find_all('a', attrs={'class':"default-btn-shortcode dt-btn dt-btn-s link-hover-off"})[:-3]
  lenght = len(links)
  for i in range(lenght):
    url = links[i].get('href')
    if url[:5] != 'https':
      url = prefix + url
    if '//collection' in url:
      url = url.replace('//collection', '/en/collection')
    watch_url.append(url)

In [ ]:
watch_url[28]

'https://arminstrom.com/en/collection/resonance/pure-resonance-salmon/'

In [ ]:
len(watch_url)

37

In [49]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def scrape_product_info(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    specific_models = []
    prices = []
    parent_models = []
    reference_numbers = []
    descriptions = []
    image_URLs = []
    marketing_names = []
    case_diameters = []
    case_thicknesses = []
    lug_to_lugs = []
    water_resistances = []
    case_backs = []
    brands = []

    results = soup.find_all('div', {'class': 'summary entry-summary'})

    for result in results:
        specific_model = result.find('h1', {'class': 'product_title entry-title'})
        price = result.find('p', class_='price')
        parent_model = soup.find('span', {'class': 'posted_in'}).find('a')
        reference_number = result.find('span', {'class': 'sku'})
        description = result.find('div', {'class': 'woocommerce-product-details__short-description'})
        brand = soup.find('div', {'id': 'site-title', 'class': 'assistive-text'})
        images = soup.find_all('div', {'class': 'iconic-woothumbs-images__slide iconic-woothumbs-images__slide-1'})

        for img in images:
            image_url = img.find('img').get('data-large_image', "No data-large_image attribute found")
            image_URLs.append(image_url)

        if specific_model:
            specific_models.append(specific_model.text.strip())
        if price:
            prices.append(price.text.strip())
        if parent_model:
            parent_models.append(parent_model.text.strip())
        if reference_number:
            reference_numbers.append(reference_number.text.strip())
        if description:
            descriptions.append(description.text.strip())
        if brand:
            brands.append(brand.text.replace('Uhren', '').strip())

        p_tags = soup.find_all('p')

    for p_tag in p_tags:
        text = p_tag.get_text()

        if re.search(r'(Limited\s*Edition|Edition\s*Limited)', text, re.IGNORECASE):
            marketing_names.append(text.strip())
        pattern = re.compile(r'.*\b(case\s*back)\b.*', flags=re.IGNORECASE)
        lines = text.strip().split('\n')
        for caseback in lines:
            caseback = re.sub(r'^[^,]*,', '', caseback)
            if re.search(pattern, caseback):
                case_backs.append(caseback.strip())

    all_text_tags = soup.find_all(string=True)
    for text_content in all_text_tags:
              if isinstance(text_content, str):
                matches = re.findall(r'\b(\s*Diameter:)\s*([^\n]*)', text_content)
                if matches:
                  for diameter in matches:
                    case_diameters.append(diameter[1].strip())

                  for text_content in all_text_tags:
                    if isinstance(text_content, str):
                      matches = re.findall(r'\b(\s*Height:)\s*([^\n]*)', text_content)
                      if matches:
                        for case_thickness in matches:
                          case_thicknesses.append(case_thickness[1].strip())

                          for text_content in all_text_tags:
                                if isinstance(text_content, str):
                                  matches = re.findall(r'\b(\s*Water-resistance:|Water\s*resistance:)\s*([^\n]*)', text_content)
                                  if matches:
                                    for water_resistance in matches:
                                      water_resistances.append(water_resistance[1].strip())

                        for text_content in all_text_tags:
                          if isinstance(text_content, str):
                            matches = re.findall(r'\b(\s*Lug-to-Lug:)\s*([^\n]*)', text_content)
                            if matches:
                              for lug_to_lug in matches:
                                lug_to_lugs.append(lug_to_lug[1].strip())


    data = {
        'Specific_Model': specific_models,
        'Price': prices,
        'Parent_Model': parent_models,
        'Reference Number': reference_numbers,
        'Description': descriptions,
        'Image URL': image_URLs,
        'Marketing Names': ", ".join(marketing_names),
        'diameter': ", ".join(case_diameters),
        'Case Thickness': ", ".join(case_thicknesses),
        'Lug-to-Lug ': ", ".join(lug_to_lugs),
        'Water_resistance': ", ".join(water_resistances),
        'Caseback': ", ".join(case_backs),
        'Watch URL': url,
        'brand' : brands
    }

    df = pd.DataFrame(data)
    df['Currency'] = df['Price'].str.extract(r'([A-Za-z]+)')
    df['Price'] = df['Price'].str.replace(r'[^0-9.,]', '', regex=True)
    df['nickname'] = df['Specific_Model']
    df['made_in'] = 'Switzerland'
    return df



all_data = pd.DataFrame()

for url in watch_url:
    df = scrape_product_info(url)
    all_data = pd.concat([all_data, df], ignore_index=True)

all_data


,Specific_Model,Price,Parent_Model,Reference Number,Description,Image URL,Marketing Names,diameter,Case Thickness,Lug-to-Lug,Water_resistance,Caseback,Watch URL,brand,Currency,nickname,made_in
0,One Week First Edition,32000,System 78,ST23-OW.FE,More than an evolution. The One Week completel...,https://arminstrom.com/wp-content/uploads/2023...,Limited Edition\n25 pieces,41.00mm,10.60 mm,46.20mm,10 ATM / 100 m,Sapphire crystal and case back with anti-refle...,https://arminstrom.com/en/collection/system-78...,Armin Strom,CHF,One Week First Edition,Switzerland
1,Orbit Manufacture Edition,35000,System 78,ST22-OR.90,Armin Strom has completely reinterpreted the u...,https://arminstrom.com/wp-content/uploads/2022...,,43.4 mm,"12,6 mm",48.96 mm,5 ATM / 50 m,Sapphire crystal and case back with anti-refle...,https://arminstrom.com/en/collection/system-78...,Armin Strom,CHF,Orbit Manufacture Edition,Switzerland
2,Orbit First Edition,35000,System 78,ST22-OR.75,Armin Strom has completely reinterpreted the u...,https://arminstrom.com/wp-content/uploads/2022...,Limited Edition\n25 pieces,43.4 mm,"12,6 mm",48.96 mm,5 ATM / 50 m,Sapphire crystal and case back with anti-refle...,https://arminstrom.com/en/collection/system-78...,Armin Strom,CHF,Orbit First Edition,Switzerland
3,Gravity Equal Force Manufacture Edition Black,20000,System 78,ST19-GEF.90,The Gravity Equal Force is the world’s first a...,https://arminstrom.com/wp-content/uploads/2021...,,41.00 mm,12.65 mm,48.09 mm,30 m,Sapphire crystal and case back with anti-refle...,https://arminstrom.com//en/collection/system-7...,Armin Strom,CHF,Gravity Equal Force Manufacture Edition Black,Switzerland
4,Gravity Equal Force Manufacture Edition Blue,20000,System 78,ST19-GEF.05,The Gravity Equal Force is the world’s first a...,https://arminstrom.com/wp-content/uploads/2021...,,41.00 mm,12.65 mm,48.09 mm,30 m,Sapphire crystal and case back with anti-refle...,https://arminstrom.com//en/collection/system-7...,Armin Strom,CHF,Gravity Equal Force Manufacture Edition Blue,Switzerland
5,Gravity Equal Force Manufacture Edition Rose Gold,31900,System 78,RG19-GEF.85,The Gravity Equal Force is the world’s first a...,https://arminstrom.com/wp-content/uploads/2021...,,41.00 mm,12.65 mm,48.09 mm,30 m,Sapphire crystal and case back with anti-refle...,https://arminstrom.com//en/collection/system-7...,Armin Strom,CHF,Gravity Equal Force Manufacture Edition Rose Gold,Switzerland
6,Gravity Equal Force Ultimate Sapphire,27500,System 78,ST21-GEF.SA,The Gravity Equal Force is the world’s first a...,https://arminstrom.com/wp-content/uploads/2021...,,41.00 mm,12.65 mm,48.09 mm,30 m,Sapphire crystal and case back with anti-refle...,https://arminstrom.com/en/collection/system-78...,Armin Strom,CHF,Gravity Equal Force Ultimate Sapphire,Switzerland
7,Gravity Equal Force Jungle Green,22900,System 78,ST22-GEF.JG.CR.M.N1,An unprecedented feat of micromechanical ingen...,https://arminstrom.com/wp-content/uploads/2022...,,41.00 mm,"12,65 mm",48.09 mm,30 m,Sapphire crystal and case back with anti-refle...,https://arminstrom.com//en/collection/system-7...,Armin Strom,CHF,Gravity Equal Force Jungle Green,Switzerland
8,Lady Beat Manufacture Edition Black,18000,System 78,ST20-LAB.90,"Finally, a no-compromise haute-horology watch ...",https://arminstrom.com/wp-content/uploads/2021...,,38.00 mm,11.65 mm,38.85 mm,30 m,Sapphire crystal and case back with anti-refle...,https://arminstrom.com//en/collection/system-7...,Armin Strom,CHF,Lady Beat Manufacture Edition Black,Switzerland
9,Lady Beat Manufacture Edition White,18000,System 78,ST20-LAB.60,"Finally, a no-compromise haute-horology watch ...",https://arminstrom.com/wp-content/uploads/2021...,,38.00 mm,11.65 mm,38.85 mm,30 m,Sapphire crystal and case back with anti-refle...,https://arminstrom.com//en/collection/system-7...,Armin Strom,CHF,Lady Beat Manufacture Edition White,Switzerland


In [18]:
all_data.head(5)

,Specific Model,Price,Parent Model,Reference Number,Description,Image URL,Marketing Names,Case Diameters,Case Thicknesses,Lug-to-Lug Dimensions,Water Resistances,Case Backs,Watch URL,brand,Currency,nickname,made_in
0,One Week First Edition,32000,System 78,ST23-OW.FE,More than an evolution. The One Week completel...,https://arminstrom.com/wp-content/uploads/2023...,Limited Edition\n25 pieces,41.00mm,10.60 mm,46.20mm,10 ATM / 100 m,Sapphire crystal and case back with anti-refle...,https://arminstrom.com/en/collection/system-78...,Armin Strom,CHF,One Week First Edition,Switzerland
1,Orbit Manufacture Edition,35000,System 78,ST22-OR.90,Armin Strom has completely reinterpreted the u...,https://arminstrom.com/wp-content/uploads/2022...,,43.4 mm,"12,6 mm",48.96 mm,5 ATM / 50 m,Sapphire crystal and case back with anti-refle...,https://arminstrom.com/en/collection/system-78...,Armin Strom,CHF,Orbit Manufacture Edition,Switzerland
2,Orbit First Edition,35000,System 78,ST22-OR.75,Armin Strom has completely reinterpreted the u...,https://arminstrom.com/wp-content/uploads/2022...,Limited Edition\n25 pieces,43.4 mm,"12,6 mm",48.96 mm,5 ATM / 50 m,Sapphire crystal and case back with anti-refle...,https://arminstrom.com/en/collection/system-78...,Armin Strom,CHF,Orbit First Edition,Switzerland
3,Gravity Equal Force Manufacture Edition Black,20000,System 78,ST19-GEF.90,The Gravity Equal Force is the world’s first a...,https://arminstrom.com/wp-content/uploads/2021...,,41.00 mm,12.65 mm,48.09 mm,30 m,Sapphire crystal and case back with anti-refle...,https://arminstrom.com//en/collection/system-7...,Armin Strom,CHF,Gravity Equal Force Manufacture Edition Black,Switzerland
4,Gravity Equal Force Manufacture Edition Blue,20000,System 78,ST19-GEF.05,The Gravity Equal Force is the world’s first a...,https://arminstrom.com/wp-content/uploads/2021...,,41.00 mm,12.65 mm,48.09 mm,30 m,Sapphire crystal and case back with anti-refle...,https://arminstrom.com//en/collection/system-7...,Armin Strom,CHF,Gravity Equal Force Manufacture Edition Blue,Switzerland


In [6]:
all_data.shape

(37, 17)

# Spec data:

In [7]:
%%time
i = 0
crystal = []
case_material = []
dial_color = []
clasp_type = []
bracelet_material = []
bracelet_color = []
between_lugs = []
numerals = []
for url in watch_url:
  r = requests.get(url)
  soup = BeautifulSoup(r.text, 'html.parser')
  p_tags = soup.find_all('p')


  for p_tag in p_tags:
    text = p_tag.get_text()
  #gettign crystal:
    if 'Case' in text:
      list_text = text.split('\n')
      for ls_text in list_text:
        if 'crystal' in ls_text:
          crystal.append(ls_text)


        # getting case_material:
    matches = re.findall(r'Case\s*([^,]*)', text)
    if matches:
      for match in matches:
          case_material.append(match.strip().split('\n')[0])




            #getting dial_color:

    matches = re.findall(r'Dial\s*(.*)', text)
    if matches:
      for Dial in matches:
          if (len(str(Dial))) > 10:
            dial_color.append(Dial.strip())
            numerals.append(Dial.strip())




    #getting clasp_type:

    matches = re.findall(r'Buckle\s*(.*)', text)
    if matches:
      for Buckle in matches:
        clasp_type.append(Buckle.strip())




    #getting bracelet_material/color :

    if "Strap" in text or "Bracelet" in text:
    # Use regular expression to find the first line after "Strap" or "Bracelet"
      Strap = re.search(r'(Strap|Bracelet).*?\n(.*?)\n', text)
      if Strap:
        bracelet_material.append(Strap.group(2).strip())
        bracelet_color.append(Strap.group(2).strip())



     #getting_ strap

    if "Strap" in text or "Bracelet" in text:
    # Use regular expression to find the desired lines
      matches = re.findall(r'(Width lugs/buckle:)\s*([^\n]*)', text)
    # if matches:
      for match in matches:
        between_lugs.append(match[1].strip())









  if url == watch_url[28]:
      dom = etree.HTML(str(soup))
      dial_color.append(str(dom.xpath("//strong[text()='Dial']/../following-sibling::div/text()")[0]).strip())
      numerals.append(str(dom.xpath("//strong[text()='Dial']/../following-sibling::div/text()")[0]).strip())
      bracelet_material.append(str(dom.xpath('//*[@id="content"]/div/section[6]/div[2]/div[2]/div/div/div/div/div[4]/text()')[0]).strip())
      bracelet_color.append(str(dom.xpath('//*[@id="content"]/div/section[6]/div[2]/div[2]/div/div/div/div/div[4]/text()')[0]).strip())
      clasp_type.append(str(dom.xpath('//*[@id="content"]/div/section[6]/div[2]/div[2]/div/div/div/div/div[4]/text()')[0]).strip())
      case_material.append(str(dom.xpath('//*[@id="content"]/div/section[6]/div[2]/div[1]/div/div/div/div/div[2]/text()')[0].split(',')[0]).strip())
      crystal.append(str(dom.xpath('//*[@id="content"]/div/section[6]/div[2]/div[1]/div/div/div/div/div[2]/text()')[0].split(',')[1]).strip())
      between_lugs.append(np.nan)

CPU times: user 6.05 s, sys: 186 ms, total: 6.23 s
Wall time: 2min 33s


In [8]:
spec_dict = {
    'crystal': crystal,
    'case_material': case_material,
    'dial_color': dial_color,
    'clasp_type': clasp_type,
    'bracelet_material': bracelet_material,
    'bracelet_color': bracelet_color,
    'between_lugs': between_lugs,
    'numerals': numerals
}

In [9]:
df_spec = pd.DataFrame({ key:pd.Series(value) for key, value in spec_dict.items() })

In [ ]:
df_spec.head(3)

In [23]:
df_spec.shape

(37, 9)

In [54]:
resultf_df = pd.concat([all_data, df_spec], axis=1)

In [55]:
resultf_df.shape

(37, 26)

In [56]:
resultf_df.head(3)

,Specific_Model,Price,Parent_Model,Reference Number,Description,Image URL,Marketing Names,diameter,Case Thickness,Lug-to-Lug,...,made_in,crystal,case_material,dial_color,clasp_type,bracelet_material,bracelet_color,between_lugs,numerals,made_in
0,One Week First Edition,32000,System 78,ST23-OW.FE,More than an evolution. The One Week completel...,https://arminstrom.com/wp-content/uploads/2023...,Limited Edition\n25 pieces,41.00mm,10.60 mm,46.20mm,...,Switzerland,"Stainless steel, Sapphire crystal and case bac...",Stainless steel,"Black minute ring and small-second dial, numbe...",Pin buckle or folding clasp available in stain...,Stainless steel with a double folding clasp,Stainless steel with a double folding clasp,22-20 mm,"Black minute ring and small-second dial, numbe...",Switzerland
1,Orbit Manufacture Edition,35000,System 78,ST22-OR.90,Armin Strom has completely reinterpreted the u...,https://arminstrom.com/wp-content/uploads/2022...,,43.4 mm,"12,6 mm",48.96 mm,...,Switzerland,Sapphire crystal and case back with anti-refle...,Stainless steel,Off-center black dial with subdial,Pin buckle or folding clasp available in stain...,Stainless-steel with a double folding clasp,Stainless-steel with a double folding clasp,22/20 mm,Off-center black dial with subdial,Switzerland
2,Orbit First Edition,35000,System 78,ST22-OR.75,Armin Strom has completely reinterpreted the u...,https://arminstrom.com/wp-content/uploads/2022...,Limited Edition\n25 pieces,43.4 mm,"12,6 mm",48.96 mm,...,Switzerland,Sapphire crystal and case back with anti-refle...,Stainless steel,Off-center dial in black gold with subdial,Pin buckle or folding clasp available in 18ct ...,Stainless-steel with a double folding clasp,Stainless-steel with a double folding clasp,22/20 mm,Off-center dial in black gold with subdial,Switzerland


In [57]:
# Add empty columns for data we didn't extract from the product PDFs.
# We could do that in the future, but for now, we will keep them blank due to time shortage.
# List of column names
resultf_df['type'] = np.nan
resultf_df['year_introduced'] = np.nan
resultf_df['style'] = np.nan
resultf_df['case_shape'] = np.nan
resultf_df['case_finish'] = np.nan
resultf_df['bezel_material'] = np.nan
resultf_df['bezel_color'] = np.nan
resultf_df['movement'] = np.nan
resultf_df['caliber'] = np.nan
resultf_df['power_reserve'] = np.nan
resultf_df['frequency'] = np.nan
resultf_df['jewels'] = np.nan
resultf_df['features'] = np.nan
resultf_df['short_description'] = np.nan
resultf_df['weight'] = np.nan


In [68]:
resultf_df.shape

(37, 41)

In [69]:
#To rearrange columns in a pandas DataFrame
resultf_df = resultf_df[['Reference Number', 'Watch URL', 'type', 'brand', 'year_introduced', 'Parent_Model', 'Specific_Model', 'nickname', 'Marketing Names', 'style', 'Currency',
                       'Price', 'Image URL', 'made_in', 'case_shape', 'case_material', 'case_finish', 'Caseback', 'diameter',
                       'between_lugs', 'Lug-to-Lug ', 'Case Thickness', 'bezel_material', 'bezel_color', 'crystal', 'Water_resistance', 'weight',
                       'dial_color', 'numerals', 'bracelet_material', 'bracelet_color', 'clasp_type', 'movement', 'caliber',
                       'power_reserve', 'frequency', 'jewels', 'features', 'Description', 'short_description']]


In [70]:
resultf_df.head(2)

,Reference Number,Watch URL,type,brand,year_introduced,Parent_Model,Specific_Model,nickname,Marketing Names,style,...,bracelet_color,clasp_type,movement,caliber,power_reserve,frequency,jewels,features,Description,short_description
0,ST23-OW.FE,https://arminstrom.com/en/collection/system-78...,NaN,Armin Strom,NaN,System 78,One Week First Edition,One Week First Edition,Limited Edition\n25 pieces,NaN,...,Stainless steel with a double folding clasp,Pin buckle or folding clasp available in stain...,NaN,NaN,NaN,NaN,NaN,NaN,More than an evolution. The One Week completel...,NaN
1,ST22-OR.90,https://arminstrom.com/en/collection/system-78...,NaN,Armin Strom,NaN,System 78,Orbit Manufacture Edition,Orbit Manufacture Edition,,NaN,...,Stainless-steel with a double folding clasp,Pin buckle or folding clasp available in stain...,NaN,NaN,NaN,NaN,NaN,NaN,Armin Strom has completely reinterpreted the u...,NaN


In [71]:
# xtract the DataFrame df into a CSV
resultf_df.to_csv('output.csv', index=False)